# Superando el reto del billón de filas con Python

# ¿Qué vamos a hacer?

* Vamos a empezar leyendo un fichero csv de distintas formas y con distintas bibliotecas.
* Vamos a ver si podemos optimizar el espacio en memoria.
* Vamos a intentar acelerar esta parte usando todo nuestro hardware.
* Vamos a comentar si un csv es lo mejor para almacenar nuestros datos.
* ...

In [ ]:
from pathlib import Path
import csv
import gzip
from io import TextIOWrapper
import shutil
import timeit

import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import duckdb
import modin.pandas as mpd
import dask.dataframe as dd
import ibis

from src.utils import generate_sample_data, generate_heatmap

ibis.options.interactive = True

## Generación de los ficheros CSV que usaremos (si no lo habéis hecho aún)

Empezamos creando un fichero más pequeño que el de 1.000.000.000 de filas porque de lo que se trata es de explicar conceptos y no tanto del ***1brc***.

## Generamos CSV (y otros formatos) del tamaño que nos permita nuestro PC (RAM)

A la hora de almacenar datos en ficheros es muy habitual utilizar el formato CSV. Es un formato textual que se puede editar fácilmente y además tiene un soporte completo en cualquier herramienta y lenguaje de tratamiento de datos. Cualquier persona que alguna vez haya trabajado con datos se habrá encontrado con ficheros CSV y aunque como veremos quizá no sea el formato más recomendable, sí que es el más usado y por este motivo lo utilizaremos en este tutorial.

A modo informativo, el ejercicio original serían 1.000.000.000 de filas por 2 columnas. Supongamos que cada dato en memoria son 64 bits. Eso sería alrededor de 15GiB de memoria. Si en alguna operación se hace alguna copia intermedia ya hablamos de picos de 30GiB, más lo que consuma el sistema operativo y otras cosas que tengáis abiertas.

* Si vuestro PC tiene una RAM de 8GiB os recomiendo que no uséis ficheros de más de 50.000.000 o 100.000.000 filas.
* Si vuestro PC tiene una RAM de 16GiB os recomiendo que no uséis ficheros de más de 100.000.000 o 200.000.000 filas.
* Si vuestro PC tiene una RAM de 32GiB os recomiendo que no uséis ficheros de más de 500.000.000 filas.
* Si vuestro PC tiene una RAM de 64 o más GiB debería ser seguro usar ficheros de 1.000.000.000 filas.

Yo voy a usar ficheros de 50.000.000 de filas para esta primera parte para que muchas cosas no se eternicen y para ir haciendo las cosas con soltura.

A lo largo de esta parte, si no lo habéis hecho ya, generaremos varios ficheros en varios formatos y con distintos tipos de compresión.

<div class="alert alert-danger">La siguiente celda es algo que deberéis modificar en caso de que queráis probar otras cosas, usar otros nombres, etc.</div>

In [ ]:
path = Path('.')
(path / 'data').mkdir(mode=0o775, exist_ok=True)
path_data = path / 'data'
filename_ = "sample50M"

In [ ]:
# Ficheros CSV de 50M de filas. 
generate_sample_data(fmt='csv', filename=f'{str(path_data)}/{filename_}')
generate_sample_data(fmt='csv', filename=f'{str(path_data)}/{filename_}', compression= 'gzip')

# Ficheros Parquet de 50M de filas. 
generate_sample_data(fmt='parquet', filename=f'{str(path_data)}/{filename_}')
generate_sample_data(fmt='parquet', filename=f'{str(path_data)}/{filename_}', compression = 'gzip')

# Ficheros Feather de 50M de filas.
generate_sample_data(fmt='feather', filename=f'{str(path_data)}/{filename_}')
# ¿Por qué no generamos el feather.zip?

Antes de ejecutar nada voy a crear un *dataframe* para ir añadiendo los resultados de las distintas pruebas y así las podremos ver todas al final y ver qué es lo que nos ha funcionado mejor y lo que nos ha funcionado peor:

In [ ]:
resultados = pd.DataFrame(
    index=('python', 'numpy', 'pandas', 'polars', 'pyarrow', 'duckdb', 'dask', 'modin+dask'),
    columns=(
        'lee csv (por defecto/engine c)', 
        'lee csv.zip (por defecto/engine c)',
        'lee csv (engine pyarrow)', 
        'lee csv.zip (engine pyarrow)',
        'lee parquet', 
        'lee parquet comprimido',
        'lee feather'
    )
)

## Leo CSV con Python

Vamos a probar con Python puro usando solo herramientas de la biblioteca estándar:

In [ ]:
filename = f"{filename_}.csv"

In [ ]:
def lee_csv(filename):
    rows = []
    with open(path_data / filename) as f:
        csv_reader = csv.reader(f)
        for row in csv_reader:
            rows.append(row)
    return rows

def lee_csv_gzip(filename):
    rows = []
    with gzip.open(path_data / f"{filename}.gz", "rb") as fzip:
        csv_reader = csv.reader(TextIOWrapper(fzip))
        for row in csv_reader:
            rows.append(row)
    return rows

In [ ]:
kk = %timeit -o -r 3 lee_csv(filename)

In [ ]:
resultados.loc['python', 'lee csv (por defecto/engine c)'] = kk.average

In [ ]:
kk = %timeit -o -r 3 lee_csv_gzip(filename)

In [ ]:
resultados.loc['python', 'lee csv.zip (por defecto/engine c)'] = kk.average

Lo anterior lo hemos metido en una lista de listas. Eso, normalmente, sería poco eficiente para luego hacer las distintas operaciones que necesitemos con los datos. Lo suyo sería meterlo en un `np.array` por lo que también habría que incluir ese tiempo en lo anterior. Lo dejamos así por simplificar.

Veamos como van los resultados, de momento:

In [ ]:
resultados.transpose()

* ¿Tiene sentido comprimir los ficheros? ¿Cuándo? ¿Por qué?,...

## Leo CSV con numpy

Vamos a probar con `numpy`. Dispone de funcionalidad para leer ficheros pero, en general, no son ninguna maravilla en lo que a rendimiento se refiere. Si hay que abrir un CSV de miles, decenas de miles o, incluso, centenas de miles de filas no es una mala opción ya que el tiempo que necesita no es tanto. Si tenemos que abrir ficheros más grandes quizá sea una mala opción. Pero no nos adelantemos. Hagamos las pruebas:

In [ ]:
kk = %timeit -o -r 3 np.genfromtxt(path_data / filename, delimiter=',')

In [ ]:
resultados.loc['numpy', 'lee csv (por defecto/engine c)'] = kk.average

In [ ]:
kk = %timeit -o -r 3 np.genfromtxt(path_data / f"{filename}.gz", delimiter=',')

In [ ]:
resultados.loc['numpy', 'lee csv.zip (por defecto/engine c)'] = kk.average

En este caso estamos usando `np.genfromtxt` que ofrece cierta flexibilidad y suele tener mejor rendimiento que `np.loadtxt`. Pero, como hemos comentado antes, `numpy` está usando mucho código python poco optimizado/que hace muchas cosas para la lectura.

Veamos como van los resultados:

In [ ]:
resultados.transpose()

* ¿Qué podemos comentar de esto?

## Leo CSV con pandas

Con pandas, ¿es más rápido abrir un fichero comprimido o un fichero sin comprimir?, ¿es más rápido que usando las opciones anteriores? Veamos:

In [ ]:
kk = %timeit -r 3 -o df = pd.read_csv(path_data / filename)

In [ ]:
resultados.loc['pandas', 'lee csv (por defecto/engine c)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = pd.read_csv(path_data / f"{filename}.gz")

In [ ]:
resultados.loc['pandas', 'lee csv.zip (por defecto/engine c)'] = kk.average

`pandas`, por defecto, usa un *motor* de lectura escrito en *C*. Además, dispone de un *motor* escrito en *Python*, que es más lento pero más flexible [1] y, desde la versión 1.4, dispone de un *motor* que usa `arrow` [2].

[1] https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas-read-csv

[2] https://pandas.pydata.org/docs/user_guide/pyarrow.html#i-o-reading

Antes hemos usado la opción por defecto, `engine='c'`. Ahora usaremos `pyarrow`:

In [ ]:
kk = %timeit -r 3 -o df = pd.read_csv(path_data / filename, engine='pyarrow')

In [ ]:
resultados.loc['pandas', 'lee csv (engine pyarrow)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = pd.read_csv(path_data / f"{filename}.gz", engine='pyarrow')

In [ ]:
resultados.loc['pandas', 'lee csv.zip (engine pyarrow)'] = kk.average

Vamos, nuevamente, a ver los resultados:

In [ ]:
resultados.transpose()

* ¿Qué hemos visto hasta ahora?

* ¿El comprimir el fichero merece la pena?

* ¿Es lo mismo usar el *engine* `c` o el *engine* `pyarrow`?

* ¿Estamos aprovechando todo nuestro hardware con `pandas`? ¿Con `pandas` + `pyarrow`?

## Leo CSV con Polars

Vamos a añadir ahora `Polars` a ver qué tal se comporta. `Polars` es parecido a `pandas` en espíritu pero tiene varias diferencias:

* Escrito en Rust.
* No usa índices.
* Usa `arrow` en lugar de `numpy` para almacenar los datos en memoria.
* ...

¿Probamos a ver qué tal?

In [ ]:
kk = %timeit -r 3 -o df = pl.read_csv(path_data / filename)

In [ ]:
resultados.loc['polars', 'lee csv (engine pyarrow)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = pl.read_csv(path_data / f"{filename}.gz")

In [ ]:
resultados.loc['polars', 'lee csv.zip (engine pyarrow)'] = kk.average

Eso parece que ha sido rápido. Veamos los resultados:

In [ ]:
resultados.transpose()

`Polars` ha llegado a su versión 1.0. Se considera estable. Por otro lado, también tiene un desarrollo muy rápido aunque todavía no dispone de toda la funcionalidad que tiene `pandas` en este momento. A veces nos puede interesar convertir un *dataframe* de `Polars` a `pandas` para seguir trabajando con `pandas`. Vamos a leer el fichero usando `Polars` y veamos varias cosas interesantes:

In [ ]:
df = pl.read_csv(path_data / filename)

In [ ]:
df.estimated_size("mb")

In [ ]:
print(df)

Si, por lo que sea, necesitamos usar alguna funcionalidad que está en `pandas` y no en `Polars` podríamos hacer lo siguiente:

In [ ]:
# use_pyarrow_extension_array=False es la opción por defecto
# Polars nos dará un dataframe de pandas con tipos de python, str, np, etc
padf = df.to_pandas()

Ahora podemos usar, por ejemplo, el método `info` que no está en `Polars`:

In [ ]:
padf.info(memory_usage='deep')

Aquí vemos dos cosas.

La primera, al transformarlo a un *dataframe* de `pandas` ha tardado algo de tiempo:

In [ ]:
%timeit df.to_pandas()

Si queremos leer con `Polars` y luego usar la información para hacer operaciones con `pandas` vemos que tarda casi un segundo en hacer la transformación. Por tanto, puede haber casos en que usar directamente `pandas` para leer sea más eficiente que usar `Polars` para leer y transformar a `pandas`.

Segundo, vemos los tamaños de ambos *dataframes*. El de `Polars` ocupa bastante menos. `Polars` usa `arrow` en sus tripas y esto se nota. Podemos hacer que `pandas` use `arrow` también:

In [ ]:
df.to_pandas(use_pyarrow_extension_array=True).info(memory_usage='deep')

* ¿Qué hemos visto ahora? Dos cositas que acabamos de comentar...

## Leo CSV con PyArrow

Hemos visto que podemos usar `PyArrow` para leer ficheros con `pandas`. Hemos visto que `Polars` usa `arrow` para almacenar los datos...

Vamos a usar `PyArrow` para leer el csv a ver lo que nos ofrece sin intermediarios:

In [ ]:
kk = %timeit -r 3 -o df = pa.csv.read_csv(path_data / filename)

In [ ]:
resultados.loc['pyarrow', 'lee csv (engine pyarrow)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = pa.csv.read_csv(path_data / f"{filename}.gz")

In [ ]:
resultados.loc['pyarrow', 'lee csv.zip (engine pyarrow)'] = kk.average

¿Cómo están ahora los resultados?

In [ ]:
resultados.transpose()

* ¿Es conveniente usar siempre `Pyarrow`?
* ¿Merece la pena el pequeño retraso que introduce `Polars` sobre `PyArrow`?
* ¿Es asumible usar `pandas`?
* ¿Debemos usar `pandas` usando `numpy` como *backend* o es mejor usar `arrow` como *backend*?

## Leo CSV con DuckDB

DuckDB es una base de datos analítica que se integra perfectamente en Python. Y que además, podemos usar como cualquier otra librería. Vamos a usarla aquí para leer el fichero en formato CSV, comprimido y sin comprimir, y lo metemos en una tabla:

In [ ]:
kk = %timeit -r 3 -o duckdb.sql(f"CREATE OR REPLACE TEMPORARY TABLE temp_table AS SELECT * FROM './{str(path_data)}/{filename}'")

In [ ]:
resultados.loc['duckdb', 'lee csv (por defecto/engine c)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o duckdb.sql(f"CREATE OR REPLACE TEMPORARY TABLE temp_table AS SELECT * FROM './{str(path_data)}/{filename}.gz'")

In [ ]:
resultados.loc['duckdb', 'lee csv.zip (por defecto/engine c)'] = kk.average

In [ ]:
resultados.transpose()

### Pequeño inciso

Existe una biblioteca que se llama `ibis`. Pretende ser una abstracción sobre muchos *backend*s: PostgreSQL, MySQL, SQLite, Polars,... El *backend* por defecto que usa es `DuckDB` y permite usar todo esto de una forma que se parece más a cómo usamos `pandas` o `Polars`. Entre las ventajas que le veo, lo que comento, que permite cambiar de *backend* sin cambiar apenas el código que usamos y que muchas expresiones se parecen más a `pandas` o `Polars` que a SQL. Vemos un pequeño ejemplo (recordad que por debajo usa `DuckDB` si no cambiamos nada):

In [ ]:
ibis.read_csv(path_data / filename).head(5)

Reproducimos lo que hemos hecho antes con `DuckDB`, leer el csv y meterlo en una tabla en memoria:

In [ ]:
%timeit -r 3 -o ibis.read_csv(path_data / filename).as_table().cache()

# ¿Qué pasa con el tipo de los datos?

En general, las distintas bibliotecas generalizan al tipo de dato que abarca mayor rango para el valor. Anteriormente hemos visto, por ejemplo:

In [ ]:
print(padf.info())
print()
print(padf['price'].min(), padf['price'].max())
print()
print(padf['product'].unique())

* ¿Necesitamos usar 'int64' para guardar datos que se encuentran en un rango entre 5 y 114?
* ¿Cabrían en otro tipo de dato?

Vamos a adaptar el código de [1] para hacer una pequeña prueba usando `numpy`. No se trata de ver lo eficiente que es `numpy` en hacer una operación concreta sino de entender la implicación de usar algún tipo de dato u otro:

[1] https://code.whatever.social/questions/15340781/python-numpy-data-types-performance

In [ ]:
import timeit

setup = """
import numpy as np
A = np.ones((1000,1000,3), dtype=datatype)
"""

datatypes = "np.uint8", "np.uint16", "np.uint32", "np.uint64",  "np.float16", "np.float32", "np.float64"

stmt1 = """
A = A * 255
A = A / 255
A = A - 1
A = A + 1
"""

stmt2 = """
A *= 255
A -= 1
A += 1
"""

stmt3 = """
A = A * 255 / 255 - 1 + 1
"""

stmt4 = """
A[:,:,:2] *= A[:,:,:2]
"""

stmt5 = """
A[:,:,:2] = A[:,:,:2] * A[:,:,:2]
"""

stmt6 = """
A *= 4
"""

for stmt in [stmt1, stmt2, stmt3, stmt4, stmt5, stmt6]:
    print(stmt)
    for d in datatypes:
        s = setup.replace("datatype", d)
        T = timeit.Timer(stmt=stmt, setup=s)
        print(d,":", min(T.repeat(number=30)))
    print()
print()

* ¿Qué conclusiones podemos obtener de los resultados anteriores?
* ¿Por qué el `float16` se comporta tan mal comparado con el resto?
* ¿Es una buena solución de compromiso usar tipos de datos más pequeños?

Veamos lo que ocuparía nuestro *dataframe* de `pandas` si usásemos `uint8` para la columna de precios y `bytes` para la columna de productos nos quedaría:

In [ ]:
print("con 64 bits en la columna de precios")
print(padf.info(memory_usage='deep'))
print()
print("con 8 bits en la columna de precios")
_padf = pd.DataFrame()
_padf['product'] = padf['product'].astype(bytes)
_padf['price'] = padf['price'].astype(np.uint8)
print(_padf.info(memory_usage='deep'))
print()
print(_padf.head())
del _padf

In [ ]:
print("Cálculo del espacio ocupado en MiB: ")
print(
    (
        50_000_000 # bytes para la columna 'product'
        + 
        50_000_000 # bytes para la columna 'price'
    )
    /
    1024 # de bytes a kiB
    / 
    1024 # de kiB a MiB
)

In [ ]:
print("Cuantas veces ocupa de más el df original: ")
print(3.1 * 1024 / 95.4)

* ¿Debemos usar siempre el tipo de dato más pequeño en el que quepa nuestro dato?
* ¿Es mejor elegir un tipo más universal si las operaciones son más eficientes?
* ¿Dependerá del tipo de operaciones que estemos haciendo?
* ¿Qué es el desbordamiento/*overflow*?

A lo hora de leer los datos podemos usar opciones para definir directamente el tipo de los datos que queremos que use en el momento de lectura y no lo tenemos que hacer a posteriori.

[1] https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html

[2] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

[3] https://docs.pola.rs/py-polars/html/reference/api/polars.read_csv.html

[4] https://arrow.apache.org/docs/python/generated/pyarrow.csv.read_csv.html

# Estamos usando de forma eficiente nuestro *hardware* para leer los datos

Muchas operaciones que hemos hecho hasta ahora no sacan todo el rendimiento de nuestro equipo pero hay formas de exprimir mejor nuestros recursos. Hay bibliotecas que nos permiten hacer esto sin mucho dolor de cabeza. Esto lo vamos a ver por encima, de momento.

Usando `Dask` podríamos hacer:

In [ ]:
%timeit dd.read_csv(path_data / filename)

In [ ]:
ddf = dd.read_csv(path_data / filename)

In [ ]:
ddf

In [ ]:
ddf.info()

No vemos nada, de momento, porque `Dask` usa evaluación perezosa (*lazy evaluation*). Hasta que no hay que usar algo no hace el trabajo. Si queremos que haga el trabajo de forma explícita lo podemos hacer así:

In [ ]:
ddf = dd.read_csv(path_data / filename).compute()

In [ ]:
ddf

In [ ]:
ddf.info(memory_usage='deep')

In [ ]:
kk = %timeit -r 3 -o df = dd.read_csv(path_data / filename).compute()

In [ ]:
resultados.loc['dask', 'lee csv (por defecto/engine c)'] = kk.average

En la ejecución de `Dask` con un fichero comprimido es interesante leer esa *issue*: https://github.com/dask/dask/issues/2554

Para leer el fichero comprimido `Dask` necesita meterlo todo en memoria. y no puede hacer lecturas en paralelo en caso de que hubiera varios ficheros CSV dentro del fichero comprimido. Si no usamos la opción `blocksize=None` nos responderá que el siguiente aviso:

<div class="alert alert-danger">
UserWarning: Warning zip compression does not support breaking apart files<br>
Please ensure that each individual file can fit in memory and<br>
use the keyword ``blocksize=None to remove this message``<br>
Setting ``blocksize=None``
</div>

https://docs.dask.org/en/stable/generated/dask.dataframe.read_csv.html#dask.dataframe.read_csv

In [ ]:
kk = %timeit -r 3 -o df = dd.read_csv(path_data / f"{filename}.gz", blocksize=None).compute()

In [ ]:
resultados.loc['dask', 'lee csv.zip (por defecto/engine c)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = dd.read_csv(path_data / filename, engine='pyarrow').compute()

In [ ]:
resultados.loc['dask', 'lee csv (engine pyarrow)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = dd.read_csv(path_data / f"{filename}.gz", engine='pyarrow', blocksize=None).compute()

In [ ]:
resultados.loc['dask', 'lee csv.zip (engine pyarrow)'] = kk.average

In [ ]:
resultados.transpose()

* ¿Qué podemos comentar de lo anterior?

Vamos a hacer lo mismo con `Modin`:

In [ ]:
kk = %timeit -r 3 -o df = mpd.read_csv(path_data / filename)

In [ ]:
resultados.loc['modin+dask', 'lee csv (por defecto/engine c)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = mpd.read_csv(path_data / f"{filename}.gz")

In [ ]:
resultados.loc['modin+dask', 'lee csv.zip (por defecto/engine c)'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = mpd.read_csv(path_data / filename, engine='pyarrow')

In [ ]:
resultados.loc['modin+dask', 'lee csv (engine pyarrow)'] = kk.average

In [ ]:
resultados.transpose()

* ¿Por qué `Modin` + `Dask` parece funcionar mejor que solo `Dask` (dependiendo del caso)?

# Usar otros formatos que no sean CSV

El formato CSV tiene algunas limitaciones que hay que tener en cuenta:
1. No tiene soporte para la definición del esquema, es decir, de los tipos de datos para cada columna que contiene.
2. Relacionado con el punto anterior, no es un formato que permita tener un esquema de datos fácilmente compatible hacia atrás o hacia adelante. Es decir, fácilmente podemos romper la compatibilidad del código que utilice el fichero CSV al que aplicamos algún cambio en sus columnas.
3. No existe un estándar claro respecto a cómo deben tratarse caracteres que actúan como separadores de filas o de final de línea.
4. No existe forma de definir como tratar valores nulos o *NaN*s.
5. ...

Como contraposición a los formatos textuales como CSV existen otros formatos que guardan los datos mediante una codificación diferente, que les permite reducir el espacio ocupado, incluir la posibilidad de definir un esquema de datos, organizar los datos de una forma diferente o incluso permitirte escoger un algoritmo de compresión a voluntad. Son los formatos binarios.

## Formato Apache Parquet

Entre este tipo de formatos se encuentra Apache Parquet (https://parquet.apache.org). Las principales características de este formato son:
1. Los datos contenidos se organizan por columnas y no por filas como en los ficheros CSV. Por ejemplo, esto permite que si necesitamos extraer una columna de un fichero Apache Parquet no sea necesario leer todos los datos como sí ocurre en CSV.
2. Además de organizar los datos por columnas, también se estructuran por grupos de filas. Es decir, en un fichero Parquet podemos tener las 100.000 primeras filas en un bloque o grupo en el que se guardan los 100.000 primeros valores de todas las columnas. En estos grupos de filas Parquet añade estadísticas básicas por columna como el valor mínimo, máximo y número de valores NULL que permiten descartar grupos de filas cuando estamos buscando algún valor en concreto.
3. También, al organizar los datos por columnas los algoritmos de compresión funcionarán mejor (principalmente si los datos están ordenados por alguna columna), porque será más probable que haya datos similares de forma secuencial, que ayuda a obtener una mayor compresión.
4. Y, finalmente, permite codificar dentro de cada fichero el formato o tipo de dato de cada columna.

https://pandas.pydata.org/pandas-docs/version/2.2/user_guide/io.html#io-parquet

## Leo Parquet con pandas

In [ ]:
filename = filename.replace("csv", "parquet")

In [ ]:
kk = %timeit -r 3 -o df = pd.read_parquet(path_data / filename)

In [ ]:
resultados.loc['pandas', 'lee parquet'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = pd.read_parquet(path_data / f"{filename}.gz")

In [ ]:
resultados.loc['pandas', 'lee parquet comprimido'] = kk.average

In [ ]:
resultados.transpose()

## Leo Parquet con Polars

In [ ]:
kk = %timeit -r 3 -o df = pl.read_parquet(path_data / filename)

In [ ]:
resultados.loc['polars', 'lee parquet'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = pl.read_parquet(path_data / f"{filename}.gz")

In [ ]:
resultados.loc['polars', 'lee parquet comprimido'] = kk.average

In [ ]:
resultados.transpose()

## Leo Parquet con PyArrow

In [ ]:
kk = %timeit -r 3 -o df = pa.parquet.read_table(path_data / filename)

In [ ]:
resultados.loc['pyarrow', 'lee parquet'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = pa.parquet.read_table(path_data / f"{filename}.gz")

In [ ]:
resultados.loc['pyarrow', 'lee parquet comprimido'] = kk.average

In [ ]:
resultados.transpose()

## Leo Parquet con DuckDB

In [ ]:
kk = %timeit -r 3 -o df = duckdb.sql(f"CREATE OR REPLACE TEMPORARY TABLE temp_table AS SELECT * FROM './{str(path_data)}/{filename}'")

In [ ]:
resultados.loc['duckdb', 'lee parquet'] = kk.average

In [ ]:
kk = %timeit -r 3 -o df = duckdb.sql(f"CREATE OR REPLACE TEMPORARY TABLE temp_table AS SELECT * FROM read_parquet('./{str(path_data)}/{filename}.gz')")

In [ ]:
resultados.loc['duckdb', 'lee parquet comprimido'] = kk.average

In [ ]:
resultados.transpose()

## Formato Feather

Feather sería una especie de formato arrow en memoria pero replicado en disco. Se supone que los tiempos de escritura y de lectura son más rápidos que con otros formatos pero, por contra, ocupa bastante más espacio en disco.

Lo de "se supone" es lo que os vais a llevar de esta parte de la charla. Al final del día los resultados dependen mucho del cómo y dónde los estemos procesando.

Otra desventaja de Feather con respecto a Parquet es que Parquet está mucho mejor soportado en muchos entornos y es un estándar de facto entre muchas aplicaciones para intercambiar información de forma eficiente y está mejor pensado para ser un formato de almacenamiento de datos duradero.

https://arrow.apache.org/docs/python/feather.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_feather.html

https://philipmay.org/blog/2024/pandas-data-format-and-compression.html

https://ao.vern.cc/questions/48083405/what-are-the-differences-between-feather-and-parquet

## Leo Feather con pandas

In [ ]:
filename = filename.replace("parquet", "feather")

In [ ]:
kk = %timeit -r 3 -o df = pd.read_feather(path_data / filename)

In [ ]:
resultados.loc['pandas', 'lee feather'] = kk.average

In [ ]:
resultados.transpose()

## Leo Feather con Polars

Como el fichero Feather está comprimido tenemos un comportamiento 'parecido' al comentado anteriormente con `Dask` y nos aparecerá un *warning*, En este caso lo dejo para que se vea de forma explícita pero para eliminarlo podéis usar la opción `memory_map=False`.

In [ ]:
kk = %timeit -r 3 -o df = pl.read_ipc(path_data / filename)
# kk = %timeit -r 3 -o df = pl.read_ipc(path_data / filename, memory_map=False)

In [ ]:
resultados.loc['polars', 'lee feather'] = kk.average

In [ ]:
resultados.transpose()

## Leo Feather con PyArrow

In [ ]:
kk = %timeit -r 3 -o df = pa.feather.read_feather(path_data / filename)

In [ ]:
resultados.loc['pyarrow', 'lee feather'] = kk.average

In [ ]:
resultados.transpose()

Con `PyArrow` podemos usar `read_feather` y lo leído lo convierte directamente en un *dataframe* de `pandas` o `read_table` y lo convierte en una tabla de `PyArrow`. En realidad, el primero usa el segundo y luego lo convierte en un *dataframe* de `pandas`. Debido a esto se tarda un poco más en leer que usando directamente `read_table`:

In [ ]:
kk = %timeit -r 3 -o df = pa.feather.read_table(path_data / filename)

In [ ]:
resultados.transpose()

Finalmente, representaremos los datos obtenidos para visualizar de manera más clara la comparación entre todos los resultados de esta sección.

In [ ]:
generate_heatmap(resultados.transpose(),'Heatmap de los tiempos de carga en memoria principal (seg.).')

## Apéndice

Aquí mostramos tiempos del código anterior ejecutado en otros PCs

---
SO: linux (Kernel: 5.15.0-122-generic x86_64)

Procesador: AMD Ryzen 9 3900X (cache: L1: 768 KiB L2: 6 MiB L3: 64 MiB)

HDD: Crucial model: CT1000BX500SSD1 size: 931.51 GiB speed: 6.0 Gb/s type: SSD serial

RAM: 16GiB DIMM DDR4 Speed 2666 MT/s (x 4) = 64 GiB

![Resultados](./images/resultados_kiko_desktop_linux_leyendo.png)

---
SO: Windows 10 Enterprise (64 bits)

Procesador: 13th Gen Intel(R) Core(TM) i7-1365U

HDD: NVMe CL4-3D512-Q11 NVMe SSSTC 512GB

RAM: 2 x 8 GB 3200 MHz DDR4-SDRAM = 16 GB 

![Resultados](./images/resultados_kiko_pccurr_windows_leyendo.png)

---
SO: macOS Sequoia (15.0)

Procesador: Apple M2 con 8 núcleos

HDD: APPLE SSD AP0512Z 512 GB

RAM: LPDDR5 16 GB

![Resultados](./images/resultados_jordi_mba_leyendo.png)

---
SO: Windows 11 Pro 2 (64 bits)

Procesador: 14th Intel(R) Core(TM) i7-14700HX  

HDD: NVMe™ TLC M.2 de 1 TB SSD 

RAM: 32 GB de RAM DDR5-4800 MHz (2 x 16 GB)

![Resultados](./images/resultados_ernesto_windows_leyendo.png)